In [ ]:
#!/usr/bin/env python
# coding: utf-8

# In[7]:


import numpy as np
from flask import Flask, request, jsonify, render_template
import pickle

app = Flask(__name__)
model = pickle.load(open('model.pkl', 'rb'))

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/predict',methods=['POST'])
def predict():
    '''
    For rendering results on HTML GUI
    '''
    int_features = [int(x) for x in request.form.values()]
    final_features = [np.array(int_features)]
    
    output = round(prediction[0], 2)

    return render_template('index.html', prediction_text='Estimated Suicide numbers {}'.format(output))

@app.route('/predict_api',methods=['POST'])
def predict_api():
    '''
    For direct API calls trought request
    '''
    data = request.get_json(force=True)
    prediction = model.predict([np.array(list(data.values()))])

    output = prediction[0]
    return jsonify(output)

if __name__ == "__main__":
    app.jinja_env.auto_reload = True
    app.config['TEMPLATES_AUTO_RELOAD'] = True
    app.run(debug=True)


# In[ ]:






In [33]:
import numpy as np
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
import pickle
from sklearn.externals import joblib
dataframe = pd.read_csv("master.csv")
#preprocessing
dataframe.drop(['country-year','suicides/100k pop'],axis =1,inplace = True )
dataframe.rename(columns={"gdp_for_year ($) ":"gdp_for_year_usd","gdp_per_capita ($)":"gdp_per_capita_usd","HDI for year":"HDI_for_year"},inplace=True)
dataframe["gdp_for_year_usd"] = dataframe["gdp_for_year_usd"].str.replace(',','').astype('int64')
dataframe = dataframe.query('year != 2016')
dataframe = dataframe[dataframe['suicides_no'] < 125]
X = dataframe.drop(['suicides_no'],axis = 1)
y = dataframe['suicides_no'].copy()
Europe = ["Albania","Russian Federation","France","Ukraine","Germany","Poland","United Kingdom","Italy","Spain","Hungary","Romania","Belgium","Belarus","Netherlands","Austria",
         "Czech Republic","Sweden","Bulgaria","Finland","Lithuania","Switzerland","Serbia",
         "Portugal","Croatia","Norway","Denmark","Slovakia","Latvia","Greece","Slovenia",
         "Turkey","Estonia","Georgia","Albania","Luxembourg","Armenia","Iceland","Montenegro",
         "Cyprus","Bosnia and Herzegovina","San Marino","Malta","Ireland"]
NorthAmerica = ["United States","Mexico","Canada","Cuba","El Salvador","Puerto Rico","Guatemala","Costa Rica","Nicaragua","Belize","Jamaica"]
SouthAmerica = ["Brazil","Colombia", "Chile","Ecuador","Uruguay","Paraguay","Argentina","Panama","Guyana","Suriname"]
MiddleEast = ["Kazakhstan","Uzbekistan","Kyrgyzstan","Israel","Turkmenistan","Azerbaijan","Kuwait","United Arab Emirates","Qatar","Bahrain","Oman"]
Asia = ["Japan","Republic of Korea", "Thailand", "Sri Lanka","Philippines","New Zealand","Australia","Singapore","Macau","Mongolia"]
def b(x):
    if x in Europe:
        return "Europe"
    elif x in NorthAmerica:
        return "NorthAmerica"
    elif x in SouthAmerica:
        return "SouthAmerica"
    elif x in MiddleEast:
        return "MiddleEast"
    else:
        return "Asia"
X['country'] = X['country'].apply(b)
# dataframe['country'] = dataframelog['country'].apply(b)

le = LabelEncoder()
#missing values
X['HDI_for_year'].fillna(X['HDI_for_year'].mean(), inplace=True)

#feature names
numeric_features = ['gdp_for_year_usd','gdp_per_capita_usd', 'population','HDI_for_year']
categorical_features = ['year','country', 'sex', 'age', 'generation']

#cat
mapping_dict={}
for col in categorical_features:
    X[col] = le.fit_transform(X[col])
    le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
    mapping_dict[col]=le_name_mapping
#num
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

forest_reg = RandomForestRegressor()
forest_reg.fit(X_train,y_train)

y_pred_test = forest_reg.predict(X_test)
y_pred_train = forest_reg.predict(X_train)
from sklearn.metrics import mean_squared_error
rms_test = np.sqrt(mean_squared_error(y_test, y_pred_test))
rms_train = np.sqrt(mean_squared_error(y_train, y_pred_train))
print('The RMSE of the test set is: ' + rms_test.astype(str))
print('The RMSE of the training set is: ' + rms_train.astype(str))
# Saving coloumns and model to disk
pickle.dump(forest_reg, open('model.pkl','wb'))

# Loading model to compare the results
model = pickle.load(open('model.pkl','rb'))
print('worked')

C:\Users\Salvin\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


The RMSE of the test set is: 10.915497922243903
The RMSE of the training set is: 4.675744930453382
worked


In [34]:
y_pred_train[:5]

array([86.4, 47.2,  6.5,  4.1, 11.9])

In [35]:
y_train[:5]

12286    84
23207    43
1132      8
16076     5
16486    11
Name: suicides_no, dtype: int64

In [36]:
X_train[:5]

,country,year,sex,age,population,HDI_for_year,gdp_for_year_usd,gdp_per_capita_usd,generation
12286,1,19,1,1,325356,0.768356,193870350137,51662,2
23207,1,24,1,1,162292,0.768356,50244793832,25872,2
1132,1,21,0,4,238745,0.768356,6384451606,2310,5
16076,1,22,1,4,42777,0.768356,7880509171,20333,5
16486,0,25,1,4,81700,0.756000,10003670690,8587,5


In [37]:
mapping_dict

{'year': {1985: 0,
  1986: 1,
  1987: 2,
  1988: 3,
  1989: 4,
  1990: 5,
  1991: 6,
  1992: 7,
  1993: 8,
  1994: 9,
  1995: 10,
  1996: 11,
  1997: 12,
  1998: 13,
  1999: 14,
  2000: 15,
  2001: 16,
  2002: 17,
  2003: 18,
  2004: 19,
  2005: 20,
  2006: 21,
  2007: 22,
  2008: 23,
  2009: 24,
  2010: 25,
  2011: 26,
  2012: 27,
  2013: 28,
  2014: 29,
  2015: 30},
 'country': {'Asia': 0,
  'Europe': 1,
  'MiddleEast': 2,
  'NorthAmerica': 3,
  'SouthAmerica': 4},
 'sex': {'female': 0, 'male': 1},
 'age': {'15-24 years': 0,
  '25-34 years': 1,
  '35-54 years': 2,
  '5-14 years': 3,
  '55-74 years': 4,
  '75+ years': 5},
 'generation': {'Boomers': 0,
  'G.I. Generation': 1,
  'Generation X': 2,
  'Generation Z': 3,
  'Millenials': 4,
  'Silent': 5}}